<a href="https://colab.research.google.com/github/ujj-kum/Google-Colab-Notebooks/blob/main/InceptionNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [2]:
X_train = X_train/255.0
X_test = X_test/255.0

In [3]:
import keras
from keras.models import Sequential, Model
from keras.layers import Add, Activation, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate, AveragePooling2D, Input
from keras.optimizers import Adam

In [7]:
def inception_block(x, filters):
    tower_1 = Conv2D(filters[0], kernel_size=(1, 1), padding='same', activation='relu')(x)
    tower_1 = Conv2D(filters[1], kernel_size=(3, 3), padding='same', activation='relu')(tower_1)

    tower_2 = Conv2D(filters[2], kernel_size=(1, 1), padding='same', activation='relu')(x)
    tower_2 = Conv2D(filters[3], kernel_size=(5, 5), padding='same', activation='relu')(tower_2)

    tower_3 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    tower_3 = Conv2D(filters[1], kernel_size=(1, 1), padding='same', activation='relu')(tower_3)

    output = concatenate([tower_1, tower_2, tower_3])
    return output

### Build the Inception Model

In [10]:
def Inception(input_shape, num_classes=10):
    inputs = Input(shape = input_shape)

    x = Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = inception_block(x, filters=[64, 96, 128, 16, 32])
    x = inception_block(x, filters=[128, 128, 192, 32, 96])
    x = MaxPooling2D((2, 2))(x)

    x = inception_block(x, filters=[192, 96, 208, 16, 48])
    x = inception_block(x, filters=[160, 112, 224, 24, 64])
    x = inception_block(x, filters=[128, 128, 256, 24, 64])
    x = inception_block(x, filters=[112, 144, 288, 32, 64])
    x = MaxPooling2D((2, 2))(x)

    x = inception_block(x, filters=[256, 160, 320, 32, 128])
    x = inception_block(x, filters=[256, 160, 320, 32, 128])
    x = inception_block(x, filters=[384, 192, 384, 48, 128])

    x = AveragePooling2D((4, 4))(x)
    x = Flatten()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [11]:
model = Inception(input_shape=(32, 32, 3), num_classes=10)

loss='sparse_categorical_crossentropy'
optimizer='adam'
metrics=['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

hist = model.fit(X_train, y_train, batch_size=64, epochs=5,
                 validation_data=(X_test, y_test),
                 shuffle=True)

Epoch 1/5
782/782 [==============================] - 64s 52ms/step - loss: 2.0241 - accuracy: 0.2171 - val_loss: 1.6737 - val_accuracy: 0.3567
Epoch 2/5
782/782 [==============================] - 36s 46ms/step - loss: 1.4801 - accuracy: 0.4531 - val_loss: 1.3059 - val_accuracy: 0.5210
Epoch 3/5
782/782 [==============================] - 37s 48ms/step - loss: 1.2238 - accuracy: 0.5552 - val_loss: 1.1558 - val_accuracy: 0.5745
Epoch 4/5
782/782 [==============================] - 37s 47ms/step - loss: 1.0685 - accuracy: 0.6149 - val_loss: 1.0010 - val_accuracy: 0.6426
Epoch 5/5
782/782 [==============================] - 37s 48ms/step - loss: 0.9458 - accuracy: 0.6616 - val_loss: 0.9302 - val_accuracy: 0.6741
